In [12]:
from datasets import load_dataset

In [13]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Found cached dataset parquet (/Users/k/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [15]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [16]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/k/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a0d7c5eb02493039.arrow


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /Users/k/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-a8465d7f0c34b3cf.arrow


In [18]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test-trainer")

In [23]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [24]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test-trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset, compute_metrics=compute_metrics
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.119135,0.510000
2,No log,0.998834,0.569000
3,No log,1.058193,0.577000


TrainOutput(global_step=375, training_loss=0.9327991536458333, metrics={'train_runtime': 597.3979, 'train_samples_per_second': 5.022, 'train_steps_per_second': 0.628, 'total_flos': 789354427392000.0, 'train_loss': 0.9327991536458333, 'epoch': 3.0})